In [3]:
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
import string as s
import re as r
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
import joblib as j
from sklearn.svm import SVC
from sklearn.metrics import  recall_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, LSTM,GlobalAveragePooling1D,GRU
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2025-05-22 11:26:52--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-05-22 11:26:53--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-05-22 11:26:53--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Language Detection.csv')
df2 = pd.read_csv('/content/drive/MyDrive/languages.csv')

In [6]:
df.shape

(10337, 2)

In [7]:
df2.shape

(763684, 2)

In [8]:
df.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


In [9]:
df2.head()

,text,language
0,ich denke es handelt sich hier um ein missvers...,german
1,ich habe tom gerade erst verlassen,german
2,tom versuchte mary nur zu ärgern,german
3,tom hat mir die hand geküsst,german
4,ich wusste dass dir das gefiele,german


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10337 entries, 0 to 10336
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      10337 non-null  object
 1   Language  10337 non-null  object
dtypes: object(2)
memory usage: 161.6+ KB


In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763684 entries, 0 to 763683
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   text      763684 non-null  object
 1   language  763684 non-null  object
dtypes: object(2)
memory usage: 11.7+ MB


In [12]:
df.isnull().sum()

,0
Text,0
Language,0


In [13]:
df2.isnull().sum()

,0
text,0
language,0


In [14]:
unique_counts = df.nunique()
print(unique_counts)

Text        10267
Language       17
dtype: int64


In [15]:
unique_counts = df2.nunique()
print(unique_counts)

text        763680
language         4
dtype: int64


In [16]:
print(df['Language'].unique())

['English' 'Malayalam' 'Hindi' 'Tamil' 'Portugeese' 'French' 'Dutch'
 'Spanish' 'Greek' 'Russian' 'Danish' 'Italian' 'Turkish' 'Sweedish'
 'Arabic' 'German' 'Kannada']


In [17]:
print(df2['language'].unique())

['german' 'french' 'spanish' 'english']


In [18]:
df['Language'].value_counts()

,count
Language,
English,1385
French,1014
Spanish,819
Portugeese,739
Italian,698
Russian,692
Sweedish,676
Malayalam,594
Dutch,546


In [19]:
df2['language'].value_counts()

,count
language,
english,275687
german,199618
french,169693
spanish,118686


In [20]:
df_upsampled = pd.DataFrame()

for language in df['Language'].unique():
    language_df = df[df['Language'] == language]

    if not language_df.empty:
        count = language_df['Language'].value_counts().values[0]

        if count < 1385:
            upsampled_df = resample(language_df,
                                    replace=True,
                                    n_samples=1385,
                                    random_state=42)
            df_upsampled = pd.concat([df_upsampled, upsampled_df])
        else:
            df_upsampled = pd.concat([df_upsampled, language_df])

In [21]:
df_upsampled.reset_index(drop=True, inplace=True)

df_upsampled['Language'].value_counts()

,count
Language,
English,1385
Malayalam,1385
Hindi,1385
Tamil,1385
Portugeese,1385
French,1385
Dutch,1385
Spanish,1385
Greek,1385


In [22]:
target_count = 50000

df_downsampled = pd.DataFrame()

for lang in df2['language'].unique():
    lang_df = df2[df2['language'] == lang]

    if len(lang_df) >= target_count:
        lang_downsampled = resample(lang_df,
                                    replace=False,
                                    n_samples=target_count,
                                    random_state=42)
        df_downsampled = pd.concat([df_downsampled, lang_downsampled], ignore_index=True)
    else:
        print(f"⚠️ Skipping '{lang}' — only {len(lang_df)} samples available (less than {target_count}).")

In [23]:
df2 = df_downsampled
df_downsampled.reset_index(drop=True, inplace=True)

df_downsampled['language'].value_counts()

,count
language,
german,50000
french,50000
spanish,50000
english,50000


In [24]:
pt=PorterStemmer()
def preprocessing(x):
    pun = s.punctuation
    t = []
    m = x.split()
    for i in m:

        word = ''.join(char for char in i if char not in pun)
        if word:
            stemmed_word = pt.stem(word)
            t.append(stemmed_word)
    return " ".join(t)

In [25]:
df_upsampled['Text'] = df_upsampled['Text'].apply(preprocessing)

In [26]:
df_downsampled['text'] = df_downsampled['text'].apply(preprocessing)

In [27]:
df_upsampled

,Text,Language
0,natur in the broadest sens is the natur physic...,English
1,natur can refer to the phenomena of the physic...,English
2,the studi of natur is a larg if not the onli p...,English
3,although human are part of natur human activ i...,English
4,1 the word natur is borrow from the old french...,English
...,...,...
23540,ಪರೀಕ್ಷೆಯಲ್ಲಿ ಒಂದೇ ಪ್ರಶ್ನೆಗೆ ಉತ್ತರಿಸಲು ಅಥವಾ ನಿಷ...,Kannada
23541,ಇದರ ಅಡಿಯಲ್ಲಿ ನನಗೆ ಈ 10 ಪದಗಳಲ್ಲಿ ಯಾವುದು ನಿಮ್ಮ ನ...,Kannada
23542,ನಿಮ್ಮನ್ನು ತೊಂದರೆಗೊಳಿಸಿದ್ದಕ್ಕೆ ಕ್ಷಮಿಸಿ,Kannada
23543,ನಾನು ಅದರ ಬಗ್ಗೆ ನಿಜವಾಗಿಯೂ ವಿಷಾದಿಸುತ್ತೇನೆ,Kannada


In [28]:
df_downsampled

,text,language
0,wir können ihnen derzeit kein weiteren einzelh...,german
1,unter diesen umständen werden wir kein hau kau...,german
2,elefanten sind pflanzenfress,german
3,ich habe nicht so viel freund wie tom,german
4,er erwartet einen anruf,german
...,...,...
199995,i have three exwiv,english
199996,i am hang up a pictur of my grandmoth,english
199997,tom doesnt know how to tie a tie,english
199998,it not my style,english


# Omar's work

In [ ]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df_upsampled['Text'])
y = df_upsampled['Language']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
svm_model1 = SVC(kernel='linear', C=1.0, random_state=42)
svm_model1.fit(X_train, y_train)
y_pred = svm_model1.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='macro'))
print("Recall:", recall_score(y_test, y_pred, average='macro'))
print("F1 Score:", f1_score(y_test, y_pred, average='macro'))

Accuracy: 0.9876831599065619
Precision: 0.988947309883886
Recall: 0.9875133808838134
F1 Score: 0.9878607383536332


In [ ]:
sample_text = ["Ciao"]
sample_vec = tfidf.transform(sample_text)
prediction = svm_model1.predict(sample_vec)
print("Predicted language:", prediction[0])

Predicted language: Italian


In [ ]:
joblib.dump(svm_model1, 'svm_model1.pkl')

['svm_model1.pkl']

In [ ]:
tfidf = TfidfVectorizer()
X2 = tfidf.fit_transform(df2['text'])
y2 = df2['language']
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [ ]:
svm_model2  = SVC(kernel='linear', C=1.0, random_state=42)
svm_model2.fit(X_train2, y_train2)
y_pred2 = svm_model2.predict(X_test2)
print("Accuracy:", accuracy_score(y_test2, y_pred2))
print("Precision:", precision_score(y_test2, y_pred2, average='macro'))
print("Recall:", recall_score(y_test2, y_pred2, average='macro'))
print("F1 Score:", f1_score(y_test2, y_pred2, average='macro'))

Accuracy: 0.996475
Precision: 0.996474219312796
Recall: 0.9964748242156043
F1 Score: 0.996467841287762


In [ ]:
sample_text2 = ["ich"]
sample_vec2 = tfidf.transform(sample_text2)
prediction2 = svm_model2.predict(sample_vec2)
print("Predicted language:", prediction2[0])

Predicted language: german


In [ ]:
joblib.dump(svm_model2, 'svm_model2.pkl')

['svm_model2.pkl']

In [ ]:
MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 100

In [ ]:
texts_up = df_upsampled['Text'].values
labels_up = df_upsampled['Language'].values

tokenizer_up = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer_up.fit_on_texts(texts_up)
sequences_up = tokenizer_up.texts_to_sequences(texts_up)
X_up = pad_sequences(sequences_up, maxlen=MAX_SEQUENCE_LENGTH)

le_up = LabelEncoder()
y_up = le_up.fit_transform(labels_up)
y_up_cat = to_categorical(y_up)

X_train_up, X_val_up, y_train_up, y_val_up = train_test_split(X_up, y_up_cat, test_size=0.2, random_state=42, stratify=y_up)

model_up = Sequential([
    Embedding(input_dim=MAX_NUM_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
    GRU(128),
    Dropout(0.5),
    Dense(y_up_cat.shape[1], activation='softmax')
])

model_up.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Training GRU model on UPSAMPLED data...")
model_up.fit(X_train_up, y_train_up, validation_data=(X_val_up, y_val_up), epochs=10, batch_size=64)

# Evaluation on UPSAMPLED
preds_up = model_up.predict(X_val_up)
y_true_up = np.argmax(y_val_up, axis=1)
y_pred_up = np.argmax(preds_up, axis=1)

print("\n--- UPSAMPLED METRICS ---")
print("Accuracy:", accuracy_score(y_true_up, y_pred_up))
print("Precision:", precision_score(y_true_up, y_pred_up, average='weighted'))
print("Recall:", recall_score(y_true_up, y_pred_up, average='weighted'))
print("F1 Score:", f1_score(y_true_up, y_pred_up, average='weighted'))

# Test prediction on UPSAMPLED
sample_text_up = "Ceci est un exemple de texte en français."
seq_up = tokenizer_up.texts_to_sequences([sample_text_up])
padded_up = pad_sequences(seq_up, maxlen=MAX_SEQUENCE_LENGTH)
pred_up = model_up.predict(padded_up)
pred_label_up = le_up.inverse_transform([np.argmax(pred_up)])
print("Test Text Prediction (Upsampled):", sample_text_up)
print("Predicted Language (Upsampled):", pred_label_up[0])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Training GRU model on UPSAMPLED data...
Epoch 1/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 112s 353ms/step - accuracy: 0.3636 - loss: 2.0390 - val_accuracy: 0.9577 - val_loss: 0.1940
Epoch 2/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 133s 323ms/step - accuracy: 0.9649 - loss: 0.1604 - val_accuracy: 0.9739 - val_loss: 0.1021
Epoch 3/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 102s 345ms/step - accuracy: 0.9752 - loss: 0.0942 - val_accuracy: 0.9745 - val_loss: 0.0882
Epoch 4/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 141s 343ms/step - accuracy: 0.9766 - loss: 0.0811 - val_accuracy: 0.9715 - val_loss: 0.0875
Epoch 5/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 140s 336ms/step - accuracy: 0.9802 - loss: 0.0689 - val_accuracy: 0.9749 - val_loss: 0.0778
Epoch 6/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 139s 325ms/step - accuracy: 0.9800 - loss: 0.0657 - val_accuracy: 0.9762 - val_loss: 0.0749
Epoch 7/10
295/295 ━━━━━━━━━━━━━━━━━━━━ 146s 338ms/step - accuracy: 0.9815 - loss: 0.0609 - val_accuracy: 0.9745 - val_loss: 0.0796
Epoch 8/10
295/295 ━━━━━━━━━━━━━━━━━

In [ ]:
model_up.save('geru1.keras')

In [ ]:
texts_down = df_downsampled['text'].values
labels_down = df_downsampled['language'].values

# Tokenization
tokenizer_down = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer_down.fit_on_texts(texts_down)
sequences_down = tokenizer_down.texts_to_sequences(texts_down)
X_down = pad_sequences(sequences_down, maxlen=MAX_SEQUENCE_LENGTH)

# Label encoding
le_down = LabelEncoder()
y_down = le_down.fit_transform(labels_down)
y_down_cat = to_categorical(y_down)

# Train-test split
X_train_down, X_val_down, y_train_down, y_val_down = train_test_split(
    X_down, y_down_cat, test_size=0.2, random_state=42, stratify=y_down
)

# Build GRU model with fewer units
model_down = Sequential([
    Embedding(input_dim=MAX_NUM_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
    GRU(64),  # fewer units to speed up
    Dropout(0.5),
    Dense(y_down_cat.shape[1], activation='softmax')
])

model_down.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train model
print("\nTraining GRU model on DOWNSAMPLED data with EarlyStopping...")
model_down.fit(
    X_train_down, y_train_down,
    validation_data=(X_val_down, y_val_down),
    epochs=5,                  # reduced epochs
    batch_size=64,
    callbacks=[early_stop]
)

# Evaluation
preds_down = model_down.predict(X_val_down)
y_true_down = np.argmax(y_val_down, axis=1)
y_pred_down = np.argmax(preds_down, axis=1)

print("\n--- DOWNSAMPLED METRICS ---")
print("Accuracy:", accuracy_score(y_true_down, y_pred_down))
print("Precision:", precision_score(y_true_down, y_pred_down, average='weighted'))
print("Recall:", recall_score(y_true_down, y_pred_down, average='weighted'))
print("F1 Score:", f1_score(y_true_down, y_pred_down, average='weighted'))

# Test case
sample_text_down = "Dies ist ein deutscher Beispielsatz."
seq_down = tokenizer_down.texts_to_sequences([sample_text_down])
padded_down = pad_sequences(seq_down, maxlen=MAX_SEQUENCE_LENGTH)
pred_down = model_down.predict(padded_down)
print("Test Text:", sample_text_down)
print("Predicted Language (Downsampled):", le_down.inverse_transform([np.argmax(pred_down)])[0])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Training GRU model on DOWNSAMPLED data with EarlyStopping...
Epoch 1/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 304s 120ms/step - accuracy: 0.9484 - loss: 0.1703 - val_accuracy: 0.9948 - val_loss: 0.0129
Epoch 2/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 322s 120ms/step - accuracy: 0.9969 - loss: 0.0080 - val_accuracy: 0.9963 - val_loss: 0.0088
Epoch 3/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 318s 119ms/step - accuracy: 0.9978 - loss: 0.0057 - val_accuracy: 0.9966 - val_loss: 0.0090
Epoch 4/5
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 320s 118ms/step - accuracy: 0.9977 - loss: 0.0055 - val_accuracy: 0.9966 - val_loss: 0.0096
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 26s 21ms/step

--- DOWNSAMPLED METRICS ---
Accuracy: 0.996275
Precision: 0.9962823526476634
Recall: 0.996275
F1 Score: 0.9962743895871555
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
Test Text: Dies ist ein deutscher Beispielsatz.
Predicted Language (Downsampled): german


In [ ]:
model_down.save('geru2.keras')

# **Ramez's work**

In [ ]:
#Naive Bayes

In [ ]:
x_nb = np.array(df_upsampled["Text"])
y_nb = np.array(df_upsampled["Language"])

cv = CountVectorizer()
X_nb = cv.fit_transform(x_nb)
X_train, X_test, y_train, y_test = train_test_split(X_nb, y_nb,
                                                    test_size=0.33,
                                                    random_state=42)

In [ ]:
model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
nb_accuracy = accuracy_score(y_test, y_pred)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {nb_accuracy}")

Precision: 0.9895022028674778
Recall: 0.9892820875144205
F1 Score: 0.9892818989387628
Accuracy: 0.9893178893178893


In [ ]:
model.score(X_test,y_test)

0.9893178893178893

In [ ]:
joblib.dump(model, 'nb_model.pkl')

['nb_model.pkl']

In [ ]:
user = input("Enter a Text: ")
data = cv.transform([user]).toarray()
output = model.predict(data)
print(output)


Enter a Text: Hi i am ramez i live in egypt
['English']


In [ ]:
x_nb2 = np.array(df_downsampled["text"])
y_nb2 = np.array(df_downsampled["language"])

cv = CountVectorizer()
X_nb2 = cv.fit_transform(x_nb2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_nb2, y_nb2,
                                                    test_size=0.33,
                                                    random_state=42)

In [ ]:
model2 = MultinomialNB()
model2.fit(X_train2,y_train2)

MultinomialNB()

In [ ]:
y_pred2 = model2.predict(X_test2)

In [ ]:
precision = precision_score(y_test2, y_pred2, average='macro')
recall = recall_score(y_test2, y_pred2, average='macro')
f1 = f1_score(y_test2, y_pred2, average='macro')
accuracy_score=model2.score(X_test2,y_test2)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy_score}")

Precision: 0.9980356858531152
Recall: 0.9980221169542616
F1 Score: 0.9980279942327512
Accuracy: 0.998030303030303


In [ ]:
joblib.dump(model2, 'nb_model2.pkl')

['nb_model2.pkl']

In [ ]:
user = input("Enter a Text: ")
data = cv.transform([user]).toarray()
output = model2.predict(data)
print(output)


Enter a Text: i am ramez i live in egypt and i went to use once
['english']


In [ ]:
#Multi-Layer Perceptron (MLP) on Word Embeddings

In [ ]:
df = df_upsampled  # or whatever your cleaned dataframe is called

texts = df["Text"].values
labels = df["Language"].values

# Encode language labels
le = LabelEncoder()
y = le.fit_transform(labels)
num_classes = len(np.unique(y))

# Tokenize the texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences
maxlen = 100
X = pad_sequences(sequences, maxlen=maxlen)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
embedding_index = {}
with open("glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs

# Create embedding matrix
embedding_dim = 100
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
model_mlp = Sequential([
    Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=True),
    GlobalAveragePooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model_mlp.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_mlp.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │     4,191,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,191,600 (15.99 MB)

 Trainable params: 4,191,600 (15.99 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model_mlp.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=10,
                    batch_size=32,
                    verbose=1)

Epoch 1/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 35s 72ms/step - accuracy: 0.2710 - loss: 2.4310 - val_accuracy: 0.8060 - val_loss: 1.3149
Epoch 2/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 42s 74ms/step - accuracy: 0.6996 - loss: 1.1765 - val_accuracy: 0.8543 - val_loss: 0.6507
Epoch 3/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 31s 66ms/step - accuracy: 0.8574 - loss: 0.6152 - val_accuracy: 0.9453 - val_loss: 0.3452
Epoch 4/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 37s 78ms/step - accuracy: 0.9067 - loss: 0.4005 - val_accuracy: 0.9387 - val_loss: 0.2611
Epoch 5/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 36s 68ms/step - accuracy: 0.9413 - loss: 0.2692 - val_accuracy: 0.9557 - val_loss: 0.1883
Epoch 6/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 41s 68ms/step - accuracy: 0.9469 - loss: 0.2179 - val_accuracy: 0.9620 - val_loss: 0.1490
Epoch 7/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 41s 67ms/step - accuracy: 0.9616 - loss: 0.1704 - val_accuracy: 0.9711 - val_loss: 0.1317
Epoch 8/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 41s 67ms/step - accuracy: 0.9641 - loss: 0.1508 - 

In [ ]:
loss, accuracy = model_mlp.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {accuracy}")
y_pred = model_mlp.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print(f"📌 Precision: {precision}")
print(f"📌 Recall:    {recall}")
print(f"📌 F1 Score:  {f1}")


✅ Test Accuracy: 0.9638988971710205
📌 Precision: 0.9693202072645147
📌 Recall:    0.9638989169675091
📌 F1 Score:  0.9648210012409658


In [ ]:
def predict_language(text, tokenizer, model_mlp, maxlen, label_encoder):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=maxlen)
    pred = model_mlp.predict(padded, verbose=0)
    pred_class = np.argmax(pred, axis=1)
    language = label_encoder.inverse_transform(pred_class)[0]
    return language

input_text= input("Enter a Text: ")
predicted_language = predict_language(input_text, tokenizer, model_mlp, maxlen, le)
print(f"\n🗣️ Predicted Language: {predicted_language}")

Enter a Text: test

🗣️ Predicted Language: Danish


In [ ]:
model_mlp.save('mlp_model.keras')

In [ ]:
df = df_downsampled  # or whatever your cleaned dataframe is called

texts = df["text"].values
labels = df["language"].values

# Encode language labels
le = LabelEncoder()
y = le.fit_transform(labels)
num_classes = len(np.unique(y))

# Tokenize the texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences
maxlen = 100
X = pad_sequences(sequences, maxlen=maxlen)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
embedding_index = {}
with open("glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs

# Create embedding matrix
embedding_dim = 100
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
model_mlp2 = Sequential([
    Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=True),
    GlobalAveragePooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model_mlp2.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_mlp2.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │     4,964,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,964,600 (18.94 MB)

 Trainable params: 4,964,600 (18.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model_mlp2.fit(X_train, y_train,
                    validation_split=0.2,
                    epochs=10,
                    batch_size=32,
                    verbose=1)

Epoch 1/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 259s 64ms/step - accuracy: 0.8655 - loss: 0.3947 - val_accuracy: 0.9868 - val_loss: 0.0382
Epoch 2/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 260s 64ms/step - accuracy: 0.9823 - loss: 0.0507 - val_accuracy: 0.9942 - val_loss: 0.0174
Epoch 3/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 274s 67ms/step - accuracy: 0.9850 - loss: 0.0422 - val_accuracy: 0.9884 - val_loss: 0.0293
Epoch 4/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 346s 73ms/step - accuracy: 0.9887 - loss: 0.0327 - val_accuracy: 0.9931 - val_loss: 0.0184
Epoch 5/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 283s 71ms/step - accuracy: 0.9910 - loss: 0.0261 - val_accuracy: 0.9906 - val_loss: 0.0232
Epoch 6/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 287s 62ms/step - accuracy: 0.9917 - loss: 0.0243 - val_accuracy: 0.9915 - val_loss: 0.0204
Epoch 7/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 272s 65ms/step - accuracy: 0.9922 - loss: 0.0214 - val_accuracy: 0.9954 - val_loss: 0.0144
Epoch 8/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 273s 67ms/step - accuracy: 

In [ ]:
loss, accuracy = model_mlp2.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {accuracy}")
y_pred = model_mlp2.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')
print(f"📌 Precision: {precision}")
print(f"📌 Recall:    {recall}")
print(f"📌 F1 Score:  {f1}")


✅ Test Accuracy: 0.9954749941825867
📌 Precision: 0.9955046978173215
📌 Recall:    0.995475
📌 F1 Score:  0.9954755769942467


In [ ]:
def predict_language(text, tokenizer, model_mlp2, maxlen, label_encoder):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=maxlen)
    pred = model_mlp2.predict(padded, verbose=0)
    pred_class = np.argmax(pred, axis=1)
    language = label_encoder.inverse_transform(pred_class)[0]
    return language

input_text= input("Enter a Text: ")
predicted_language = predict_language(input_text, tokenizer, model_mlp2, maxlen, le)
print(f"\n🗣️ Predicted Language: {predicted_language}")

Enter a Text: i am tall and twenty eight years old

🗣️ Predicted Language: english


In [ ]:
model_mlp2.save('mlp_model2.keras')

# Mostafa work

In [ ]:
#logestic regression for seconde dataset

In [29]:
x=df_downsampled['text']
y=df_downsampled['language']

In [30]:
X_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [31]:
vector=TfidfVectorizer()
X_train_tidf=vector.fit_transform(X_train)
x_test_tidf=vector.transform(x_test)

In [33]:
lr2=LogisticRegression()
lr2.fit(X_train_tidf,y_train)

LogisticRegression()

In [36]:
y_pred=lr2.predict(x_test_tidf)
y_tpre=lr2.predict(X_train_tidf)

In [37]:
from sklearn.metrics import classification_report,accuracy_score,precision_score,f1_score,recall_score
print("The Training Data Score....")
print("accuracy_score: ",accuracy_score(y_train,y_tpre))
print("precision_score: ",precision_score(y_train,y_tpre,average="weighted"))
print("F1_score",f1_score(y_train,y_tpre, average="weighted"))
print("recall_score",recall_score(y_train,y_tpre, average="weighted"))

The Training Data Score....
accuracy_score:  0.998275
precision_score:  0.9982761782487559
F1_score 0.9982751705124258
recall_score 0.998275


In [38]:
print("The Test Data Score....")
print("accuracy_score: ",accuracy_score(y_test,y_pred))
print("precision_score: ",precision_score(y_test,y_pred,average="weighted"))
print("F1_score",f1_score(y_test,y_pred, average="weighted"))
print("recall_score",recall_score(y_test,y_pred, average="weighted"))

The Test Data Score....
accuracy_score:  0.9947
precision_score:  0.9947106563188652
F1_score 0.9947010102488141
recall_score 0.9947


In [39]:
joblib.dump(lr2, 'lr_model2.pkl')

['lr_model2.pkl']

In [41]:
# test case
sentence1 = "Hallo, wie geht es dir?"
preprocessed_sentence1 = preprocessing(sentence1)
X_sentence1 = vector.transform([preprocessed_sentence1])
prediction1 = lr2.predict(X_sentence1)
print(f"Sentence: '{sentence1}' -> Predicted Language: {prediction1[0]}")


Sentence: 'Hallo, wie geht es dir?' -> Predicted Language: german


In [ ]:
# logistic regression for first dataset

In [ ]:
x=df_upsampled['Text']
y=df_upsampled['Language']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
vector=TfidfVectorizer()
x_train_tidf=vector.fit_transform(x_train)
x_test_tidf=vector.transform(x_test)

In [ ]:
lr=LogisticRegression()
lr.fit(x_train_tidf,y_train)

LogisticRegression()

In [ ]:
y_pred=lr.predict(x_test_tidf)
y_tpre=lr.predict(x_train_tidf)

In [ ]:
y_train_pred = lr.predict(x_train_tidf)
y_test_pred = lr.predict(x_test_tidf)
print("Training Set Metrics:")
print("Accuracy:", accuracy_score(y_train, y_train_pred))
print("Precision:", precision_score(y_train, y_train_pred, average='weighted'))
print("Recall:", recall_score(y_train, y_train_pred, average='weighted'))
print("F1-score:", f1_score(y_train, y_train_pred, average='weighted'))
print("-" * 30)


Training Set Metrics:
Accuracy: 0.9956466341049055
Precision: 0.9958275097601236
Recall: 0.9956466341049055
F1-score: 0.9956753354105485
------------------------------


In [ ]:
print("Test Set Metrics:")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Precision:", precision_score(y_test, y_test_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_test_pred, average='weighted'))
print("F1-score:", f1_score(y_test, y_test_pred, average='weighted'))

Test Set Metrics:
Accuracy: 0.9887449564663411
Precision: 0.9899168275620958
Recall: 0.9887449564663411
F1-score: 0.9890094124150913


In [ ]:
sentence4 = "Este é um exemplo de frase em português."
preprocessed_sentence4 = preprocessing(sentence4)
X_sentence4 = vector.transform([preprocessed_sentence4])
prediction4 = lr.predict(X_sentence4)
print(f"Sentence: '{sentence4}' -> Predicted Language: {prediction4[0]}")

Sentence: 'Este é um exemplo de frase em português.' -> Predicted Language: Portugeese


In [ ]:
joblib.dump(lr, 'lr_model.pkl')

['lr_model.pkl']

In [ ]:
#deep learning for the first data set

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import string as s
from nltk.stem import PorterStemmer
pt = PorterStemmer()
def preprocessing(x):
    pun = s.punctuation
    t = []
    m = x.split()
    for i in m:
        word = ''.join(char for char in i if char not in pun)
        if word:
            stemmed_word = pt.stem(word)
            t.append(stemmed_word)
    return " ".join(t)

df_upsampled['Text'] = df_upsampled['Text'].apply(preprocessing)
X = df_upsampled['Text']
y = df_upsampled['Language']
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)
max_length = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post', truncating='post')
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 16
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', #
              metrics=['accuracy'])

epochs = 10
batch_size = 32
history = model.fit(X_train_padded, y_train_encoded, epochs=epochs, batch_size=batch_size, validation_split=0.2)
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
print("Test Set Metrics:")
print("Accuracy:", accuracy_score(label_encoder.inverse_transform(y_test_encoded), y_pred))
print("Precision:", precision_score(label_encoder.inverse_transform(y_test_encoded), y_pred, average='weighted'))
print("Recall:", recall_score(label_encoder.inverse_transform(y_test_encoded), y_pred, average='weighted'))
print("F1-score:", f1_score(label_encoder.inverse_transform(y_test_encoded), y_pred, average='weighted'))


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


471/471 ━━━━━━━━━━━━━━━━━━━━ 12s 21ms/step - accuracy: 0.2493 - loss: 2.3768 - val_accuracy: 0.9246 - val_loss: 0.4431
Epoch 2/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.9344 - loss: 0.3335 - val_accuracy: 0.9676 - val_loss: 0.1641
Epoch 3/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.9712 - loss: 0.1385 - val_accuracy: 0.9748 - val_loss: 0.1018
Epoch 4/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.9748 - loss: 0.0987 - val_accuracy: 0.9782 - val_loss: 0.0807
Epoch 5/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9788 - loss: 0.0766 - val_accuracy: 0.9774 - val_loss: 0.0854
Epoch 6/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9795 - loss: 0.0712 - val_accuracy: 0.9804 - val_loss: 0.0709
Epoch 7/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.9780 - loss: 0.0690 - val_accuracy: 0.9790 - val_loss: 0.0685
Epoch 8/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9819 - loss: 0.0584 - val_accuracy: 0.9

In [ ]:
#test case
complex_sentence8_es = "La evolución de las lenguas a lo largo del tiempo es un tema fascinante para los lingüistas."
preprocessed_sentence8_es = preprocessing(complex_sentence8_es)
X_sentence8_es = tokenizer.texts_to_sequences([preprocessed_sentence8_es])
X_sentence8_es_padded = pad_sequences(X_sentence8_es, maxlen=max_length, padding='post', truncating='post')
prediction8_es = model.predict(X_sentence8_es_padded)
predicted_language_es = label_encoder.inverse_transform([np.argmax(prediction8_es)])
print(f"Sentence: '{complex_sentence8_es}' -> Predicted Language: {predicted_language_es[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Sentence: 'La evolución de las lenguas a lo largo del tiempo es un tema fascinante para los lingüistas.' -> Predicted Language: Spanish


In [ ]:
model.save('custom_cnn_model.keras')

In [ ]:
#deep learning for the seconde dataset

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import string as s
from nltk.stem import PorterStemmer

pt = PorterStemmer()
def preprocessing(x):
    pun = s.punctuation
    t = []
    m = x.split()
    for i in m:
        word = ''.join(char for char in i if char not in pun)
        if word:
            stemmed_word = pt.stem(word)
            t.append(stemmed_word)
    return " ".join(t)


df_downsampled['text'] = df_downsampled['text'].apply(preprocessing)
X2 = df_downsampled['text']
y2 = df_downsampled['language']
label_encoder = LabelEncoder()
y_encoded2 = label_encoder.fit_transform(y2)
X_train2, X_test2, y_train_encoded2, y_test_encoded2 = train_test_split(X2, y_encoded2, test_size=0.2, random_state=42)
max_words = 10000
tokenizer2 = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer2.fit_on_texts(X_train2)
X_train_sequences2 = tokenizer2.texts_to_sequences(X_train2)
X_test_sequences2 = tokenizer2.texts_to_sequences(X_test2)
max_length = 100
X_train_padded2 = pad_sequences(X_train_sequences2, maxlen=max_length, padding='post', truncating='post')
X_test_padded2 = pad_sequences(X_test_sequences2, maxlen=max_length, padding='post', truncating='post')
vocab_size = len(tokenizer2.word_index) + 1
embedding_dim = 16
model2 = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


epochs = 10
batch_size = 32
history2 = model2.fit(X_train_padded2, y_train_encoded2, epochs=epochs, batch_size=batch_size, validation_split=0.2)
loss2, accuracy2 = model2.evaluate(X_test_padded2, y_test_encoded2)
print(f"Test Loss: {loss2}")
print(f"Test Accuracy: {accuracy2}")
y_pred_encoded2 = np.argmax(model2.predict(X_test_padded2), axis=-1)
y_pred2 = label_encoder.inverse_transform(y_pred_encoded2)



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 53s 13ms/step - accuracy: 0.9182 - loss: 0.2277 - val_accuracy: 0.9966 - val_loss: 0.0098
Epoch 2/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 51s 13ms/step - accuracy: 0.9974 - loss: 0.0072 - val_accuracy: 0.9967 - val_loss: 0.0078
Epoch 3/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 82s 13ms/step - accuracy: 0.9976 - loss: 0.0057 - val_accuracy: 0.9965 - val_loss: 0.0085
Epoch 4/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 80s 12ms/step - accuracy: 0.9980 - loss: 0.0047 - val_accuracy: 0.9956 - val_loss: 0.0123
Epoch 5/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 81s 12ms/step - accuracy: 0.9981 - loss: 0.0045 - val_accuracy: 0.9966 - val_loss: 0.0078
Epoch 6/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 81s 12ms/step - accuracy: 0.9980 - loss: 0.0046 - val_accuracy: 0.9964 - val_loss: 0.0087
Epoch 7/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 49s 12ms/step - accuracy: 0.9981 - loss: 0.0042 - val_accuracy: 0.9965 - val_loss: 0.0089
Epoch 8/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 51s 13ms/step - accuracy: 0.9982 -

In [ ]:
print("Test Set Metrics:")
y_test_original = label_encoder.inverse_transform(y_test_encoded2)
y_pred_original = label_encoder.inverse_transform(y_pred_encoded2)
print("Accuracy:", accuracy_score(y_test_original, y_pred_original))
print("Precision:", precision_score(y_test_original, y_pred_original, average='weighted'))
print("Recall:", recall_score(y_test_original, y_pred_original, average='weighted'))
print("F1-score:", f1_score(y_test_original, y_pred_original, average='weighted'))

Test Set Metrics:
Accuracy: 0.99655
Precision: 0.99655768033374
Recall: 0.99655
F1-score: 0.9965515854866875


In [ ]:
model2.save('custom_cnn_model2.keras')

In [ ]:
#test case for the seconde dataset

In [ ]:
sentnce="He embarked on a quixotic quest for the mythical city of El Dorado."
preprocessed_sentence = preprocessing(sentnce)
X_sentence = tokenizer2.texts_to_sequences([preprocessed_sentence])
X_sentence_padded = pad_sequences(X_sentence, maxlen=max_length, padding='post', truncating='post')
prediction = model2.predict(X_sentence_padded)
predicted_language = label_encoder.inverse_transform([np.argmax(prediction)])
print(f"Sentence: '{sentnce}' -> Predicted Language: {predicted_language[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Sentence: 'He embarked on a quixotic quest for the mythical city of El Dorado.' -> Predicted Language: english


# **Akram's work**

# RidgeClassifier

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
X = df_upsampled['Text']
y = df_upsampled['Language']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipeline = make_pipeline(
    TfidfVectorizer(analyzer='char', ngram_range=(2, 4)),
    RidgeClassifier()
)


pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9976640475684859
              precision    recall  f1-score   support

      Arabic       1.00      1.00      1.00       280
      Danish       1.00      0.99      0.99       267
       Dutch       1.00      1.00      1.00       282
     English       1.00      0.98      0.99       252
      French       1.00      1.00      1.00       264
      German       1.00      1.00      1.00       286
       Greek       1.00      1.00      1.00       265
       Hindi       1.00      1.00      1.00       311
     Italian       0.99      0.99      0.99       281
     Kannada       1.00      1.00      1.00       279
   Malayalam       1.00      1.00      1.00       274
  Portugeese       0.99      1.00      1.00       285
     Russian       1.00      1.00      1.00       274
     Spanish       1.00      1.00      1.00       294
    Sweedish       1.00      1.00      1.00       270
       Tamil       1.00      1.00      1.00       285
     Turkish       1.00      1.00      1.00       26

In [ ]:
y_pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='macro'))
print("Recall:", recall_score(y_test, y_pred, average='macro'))
print("F1 Score:", f1_score(y_test, y_pred, average='macro'))

Accuracy: 0.9976640475684859
Precision: 0.9976813796113279
Recall: 0.9975655001794328
F1 Score: 0.997617845990271


In [ ]:
sample_text = "Yarın görüşürüz"
sample_text2= "اهلا"
sample_text3= "Hello"
predicted_language = pipeline.predict([sample_text])[0]
predicted_language2 = pipeline.predict([sample_text2])[0]
predicted_language3 = pipeline.predict([sample_text3])[0]
print("Predicted Language:", predicted_language)
print("Predicted Language:", predicted_language2)
print("Predicted Language:", predicted_language3)

Predicted Language: Turkish
Predicted Language: Arabic
Predicted Language: English


In [ ]:
joblib.dump(pipeline, 'ridge1.pkl')

['ridge1.pkl']

In [ ]:
X = df_downsampled['text']
y = df_downsampled['language']

X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, random_state=42)


pipeline2 = make_pipeline(
    TfidfVectorizer(analyzer='char', ngram_range=(2, 4)),
    RidgeClassifier()
)


pipeline2.fit(X_train2, y_train2)
y_pred2 = pipeline2.predict(X_test2)

print("Accuracy:", accuracy_score(y_test2, y_pred2))
# print(classification_report(y_test2, y_pred2))

Accuracy: 0.999425


In [ ]:
y_pred2 = pipeline2.predict(X_test2)

print("Accuracy:", accuracy_score(y_test2, y_pred2))
print("Precision:", precision_score(y_test2, y_pred2, average='macro'))
print("Recall:", recall_score(y_test2, y_pred2, average='macro'))
print("F1 Score:", f1_score(y_test2, y_pred2, average='macro'))

Accuracy: 0.999425
Precision: 0.9994249996280775
Recall: 0.9994242917277362
F1 Score: 0.9994246295340838


In [ ]:
sample_text2 = "Hola"
predicted_language2 = pipeline2.predict([sample_text2])[0]
print("Predicted Language:", predicted_language2)

Predicted Language: spanish


In [ ]:
joblib.dump(pipeline2, 'ridge2.pkl')

['ridge2.pkl']

# Char-level CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder


max_length = 200
vocab_size = 100


tokenizer = Tokenizer(char_level=True, num_words=vocab_size, lower=True)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length)

X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length)


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


num_classes = len(label_encoder.classes_)


model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=max_length),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_pad, y_train_encoded, validation_data=(X_test_pad, y_test_encoded), epochs=5)

y_pred_probs = model.predict(X_test_pad)
y_pred_classes = y_pred_probs.argmax(axis=1)

accuracy = accuracy_score(y_test_encoded, y_pred_classes)

precision = precision_score(y_test_encoded, y_pred_classes, average='macro')
recall = recall_score(y_test_encoded, y_pred_classes, average='macro')
f1 = f1_score(y_test_encoded, y_pred_classes, average='macro')

print(f"\nTest Accuracy: {accuracy}")
print(f"Macro Precision: {precision}")
print(f"Macro Recall:    {recall}")
print(f"Macro F1 Score:  {f1}")


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


589/589 ━━━━━━━━━━━━━━━━━━━━ 33s 51ms/step - accuracy: 0.6484 - loss: 1.3482 - val_accuracy: 0.9380 - val_loss: 0.1995
Epoch 2/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 40s 51ms/step - accuracy: 0.9599 - loss: 0.1354 - val_accuracy: 0.9771 - val_loss: 0.0871
Epoch 3/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 27s 45ms/step - accuracy: 0.9833 - loss: 0.0620 - val_accuracy: 0.9786 - val_loss: 0.0706
Epoch 4/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 43s 49ms/step - accuracy: 0.9932 - loss: 0.0292 - val_accuracy: 0.9875 - val_loss: 0.0459
Epoch 5/5
589/589 ━━━━━━━━━━━━━━━━━━━━ 43s 53ms/step - accuracy: 0.9963 - loss: 0.0180 - val_accuracy: 0.9786 - val_loss: 0.0673
148/148 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step

Test Accuracy: 0.9785517094924613
Macro Precision: 0.980209100779677
Macro Recall:    0.9778528209762676
Macro F1 Score:  0.9780724587958998


In [ ]:
sample_text = "كبف الحال"

sample_seq = tokenizer.texts_to_sequences([sample_text])
sample_pad = pad_sequences(sample_seq, maxlen=max_length)

pred_probs = model.predict(sample_pad)
predicted_class_index = pred_probs.argmax(axis=1)[0]

predicted_language = label_encoder.inverse_transform([predicted_class_index])[0]

print("Predicted Language:", predicted_language)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted Language: Arabic


In [ ]:
model.save('char_cnn_model1.keras')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

max_length = 100
vocab_size = 5000
embedding_dim = 50

tokenizer = Tokenizer(num_words=vocab_size, lower=True, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train2)

X_train_seq2 = tokenizer.texts_to_sequences(X_train2)
X_test_seq2 = tokenizer.texts_to_sequences(X_test2)

X_train_pad2 = pad_sequences(X_train_seq2, maxlen=max_length, padding='post', truncating='post')
X_test_pad2 = pad_sequences(X_test_seq2, maxlen=max_length, padding='post', truncating='post')

label_encoder2 = LabelEncoder()
y_train_encoded2 = label_encoder2.fit_transform(y_train2)
y_test_encoded2 = label_encoder2.transform(y_test2)
num_classes2 = len(label_encoder2.classes_)

model2 = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    Conv1D(64, kernel_size=5, activation='relu'),
    BatchNormalization(),
    GlobalMaxPooling1D(),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(num_classes2, activation='softmax')
])

model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model2.fit(X_train_pad2,y_train_encoded2,validation_data=(X_test_pad2, y_test_encoded2),epochs=10,batch_size=32,callbacks=[early_stop])

y_pred_probs2 = model2.predict(X_test_pad2)
y_pred_classes2 = y_pred_probs2.argmax(axis=1)

accuracy2 = accuracy_score(y_test_encoded2, y_pred_classes2)
precision2 = precision_score(y_test_encoded2, y_pred_classes2, average='macro', zero_division=0)
recall2 = recall_score(y_test_encoded2, y_pred_classes2, average='macro', zero_division=0)
f1_2 = f1_score(y_test_encoded2, y_pred_classes2, average='macro', zero_division=0)

print(f"\nTest Accuracy:   {accuracy2}")
print(f"Macro Precision: {precision2}")
print(f"Macro Recall:    {recall2}")
print(f"Macro F1 Score:  {f1_2}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 120s 23ms/step - accuracy: 0.9433 - loss: 0.1415 - val_accuracy: 0.9944 - val_loss: 0.0152
Epoch 2/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 142s 23ms/step - accuracy: 0.9948 - loss: 0.0136 - val_accuracy: 0.9944 - val_loss: 0.0155
Epoch 3/10
5000/5000 ━━━━━━━━━━━━━━━━━━━━ 138s 23ms/step - accuracy: 0.9953 - loss: 0.0121 - val_accuracy: 0.9934 - val_loss: 0.0159
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step

Test Accuracy:   0.994425
Macro Precision: 0.9944222866274526
Macro Recall:    0.9944232924148151
Macro F1 Score:  0.9944136611379578


In [ ]:
sample_text2 = "Bonjour"

sample_seq2 = tokenizer.texts_to_sequences([sample_text2])
sample_pad2 = pad_sequences(sample_seq2, maxlen=max_length)

pred_probs2 = model2.predict(sample_pad2)
predicted_class_index2 = pred_probs2.argmax(axis=1)[0]

predicted_language2 = label_encoder2.inverse_transform([predicted_class_index2])[0]

print("Predicted Language:", predicted_language2)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Predicted Language: spanish


In [ ]:
model2.save('char_cnn_model2.keras')